In [37]:
# Import dependancies
import numpy as np
import os
import pickle
from random import randint
import matplotlib.pyplot as plt

In [34]:
# Define data paths and size
# Load results from clustering

clustering_feature_names = ['MON',  'PRECIP' , 'AREA', 'DAILYCN', 'SNOMELT' ,'SNOFALL', 'SURQ_GEN', 'LATQ', 'WYLD', 'PET']
data_path = '/Users/muneeza/Documents/GitHub/DATA_SMest/HRU_Clustering_results/'

# all_data = np.load(data_path+'hru_clustering_data.npy')
X_train_norm = np.load(data_path+'norm_hru_clustering_data.npy')
dist = np.load(data_path+'hru_cluster_distance.npy')
centroid = np.load(data_path+'cluster_centers.npy')
labels = np.load(data_path+'hru_cluster_labels.npy')
with open(data_path+'names_list_clustering.pkl', 'rb') as f:
    names_list = pickle.load(f)

n_hrus = np.array([x.split('.')[-2] for x in names_list]).astype(int)
hrus_total = np.sum(n_hrus)

mean_t = np.mean(X_train_norm,0)
var_t = np.var(X_train_norm,0)

In [48]:
# save new names list with hru ids 
hru_names_list = []
for i, name in enumerate(names_list):
    for j in range(n_hrus[i]):
        hru_names_list.append(name.split('_')[0]+'.'+str(n_hrus[i])+'.'+str(j))

pickle.dump(names_list, open(data_path+'hru_names_list_clustering.pkl', 'wb'))

In [24]:
# # Visualize all clustered Data 

# color = []
# n = 12 # number of clusters
# for i in range(n):
#     color.append('#%06X' % randint(0, 0xFFFFFF))

# # Generate Subplot
# fig, ax = plt.subplots(3,3,figsize=(10,10))
# for i in range(3):
#     for j in range(3):
#         if i*3+j+1> 9 : # pass the others that we can't fill
#             continue
#         for k in range(n_sample):
#             ax[i, j].plot(np.transpose(X_train_norm[k,:,i*3+j+1]), color=color[labels[k]], linewidth=0.5)
#             ax[i, j].set_title(clustering_feature_names[i*3+j+1])
# plt.show()

(12, 12, 10)

In [107]:
# Sample a subset of data that is 'close enough' to original data - error in inertia < t 

n_cluster = 12
inertia = np.zeros(n_cluster)
inertia_sub = np.zeros(n_cluster)
names_list_cluster = [None]*n_cluster
names_list_sub_cluster = [None]*n_cluster

for i in range(n_cluster):
    
    id_cluster = np.argwhere(labels == i)[:,0] 
    inertia[i] = np.sum(dist[id_cluster,i])
    names_list_cluster[i] = [hru_names_list[x] for x in id_cluster]

    hrus_in_cluster = len(id_cluster)
    hrus_in_sub_cluster = int(hrus_in_cluster*0.2)  # 10% of original data 

    err_min = 200000

    for j in range (100000):
        sample = np.random.uniform(0,hrus_in_cluster,hrus_in_sub_cluster).astype(int)
        sample_id = id_cluster[sample]
        sample_inertia = np.sum(dist[sample_id,i])
        err_new = np.abs(inertia[i]-sample_inertia)
        if err_new < err_min:
            err_min = err_new
            id_min_cluster = sample_id
            inertia_sub[i] = sample_inertia

    names_list_sub_cluster[i] = [hru_names_list[x] for x in id_min_cluster]
    print('min err in inertia:' , err_min )
    np.save(data_path+'dist.in.cluster_'+str(i),dist[id_min_cluster])

pickle.dump(names_list_sub_cluster, open(data_path+'names_list_sub_clusterwise.pkl', 'wb'))
pickle.dump(names_list_cluster, open(data_path+'names_list_clusterwise.pkl', 'wb'))
    

min err in inertia: 3566.1470389062515
min err in inertia: 1772.6207281498946
min err in inertia: 1620.699867847421
min err in inertia: 3515.9254997760318
min err in inertia: 2215.908829697929
min err in inertia: 4056.403579246665
min err in inertia: 879.7470199048904
min err in inertia: 1723.9483434157498
min err in inertia: 4331.763716158501
min err in inertia: 109.40829916388688
min err in inertia: 4111.834532797489
min err in inertia: 3791.311107249996
